# Лабораторная работа 1. Методы решения задач линейной алгебры

## 1.1 LU-разложение

In [21]:
import os

import numpy as np

# Чтение входной матрицы
with open("input/1_1.txt", "r") as f:
    A_ext = np.array(
        [list(map(float, line.strip().split())) for line in f if line.strip()]
    )

# Создание папки output, если нет
os.makedirs("output", exist_ok=True)

A = A_ext[:, :-1]
b = A_ext[:, -1]
n = A.shape[0]

output = ""
print("Матрица A:")
print(A)
output += "Матрица A:\n"
output += f"{A}\n\n"

print("\nВектор b:")
print(b)
output += "Вектор b:\n"
output += f"{b}\n"

# LU-разложение с частичным выбором ведущего элемента
U = A.copy().astype(float)
L = np.eye(n)
P = np.eye(n)
pivot_amount = 0

for k in range(n - 1):
    pivot = np.argmax(np.abs(U[k:, k])) + k
    if pivot != k:
        U[[k, pivot], k:] = U[[pivot, k], k:]
        if k > 0:
            L[[k, pivot], :k] = L[[pivot, k], :k]
        P[[k, pivot], :] = P[[pivot, k], :]
        pivot_amount += 1

    if U[k, k] == 0:
        raise ValueError("Матрица вырождена")

    for i in range(k + 1, n):
        L[i, k] = U[i, k] / U[k, k]
        U[i, k:] -= L[i, k] * U[k, k:]

print("\nМатрица L:")
print(L)
output += "Матрица L:\n"
output += f"{L}\n\n"

print("\nМатрица U:")
print(U)
output += "Матрица U:\n"
output += f"{U}\n"

# Решение Lz = b (прямая подстановка)
b_permuted = P @ b
z = np.zeros(n)
for i in range(n):
    z[i] = b_permuted[i] - np.dot(L[i, :i], z[:i])

# Решение Ux = z (обратная подстановка)
x = np.zeros(n)
for i in range(n - 1, -1, -1):
    x[i] = (z[i] - np.dot(U[i, i + 1 :], x[i + 1 :])) / U[i, i]

# Определитель матрицы A
det_A = np.linalg.det(A)
print("\nОпределитель матрицы A:")
print(f"{det_A:.2f}")
output += "Определитель матрицы A:\n"
output += f"{det_A:.2f}\n\n"

# Обратная матрица
try:
    A_inv = np.linalg.inv(A)
    print("\nОбратная матрица A:")
    print(A_inv)
    output += "Обратная матрица A:\n"
    output += f"{A_inv}\n\n"
except np.linalg.LinAlgError:
    print("\nОбратная матрица не существует (матрица вырождена)")
    output += "Обратная матрица не существует (матрица вырождена)\n\n"

# Проверка решения
r = A @ x - b  # Вектор невязки
residual_norm = np.linalg.norm(r)

print("\nПроверка решения (Ax - b):")
print(r)
print(f"\nНорма невязки: {residual_norm:.2e}")
if residual_norm < 1e-8:
    print("Невязка минимальна, решение корректно!")
else:
    print("Обнаружена значительная невязка!")

output += "Проверка решения (Ax - b):\n"
output += f"{r}\n\n"
output += f"Норма невязки: {residual_norm:.2e}\n"
output += (
    "Невязка минимальна, решение корректно!\n"
    if residual_norm < 1e-8
    else "Обнаружена значительная невязка!\n"
)

# Запись вывода в файл
with open("output/1_1.txt", "w", encoding="utf-8") as f:
    f.write(output)

Матрица A:
[[ 1.  2. -2.  6.]
 [-3. -5. 14. 13.]
 [ 1.  2. -2. -2.]
 [-2. -4.  5. 10.]]

Вектор b:
[24. 41.  0. 20.]

Матрица L:
[[ 1.          0.          0.          0.        ]
 [ 0.66666667  1.          0.          0.        ]
 [-0.33333333 -0.5         1.          0.        ]
 [-0.33333333 -0.5         1.          1.        ]]

Матрица U:
[[-3.         -5.         14.         13.        ]
 [ 0.         -0.66666667 -4.33333333  1.33333333]
 [ 0.          0.          0.5         3.        ]
 [ 0.          0.          0.          8.        ]]

Определитель матрицы A:
8.00

Обратная матрица A:
[[-11.5    -2.     42.5    18.   ]
 [  5.125   1.    -18.125  -8.   ]
 [ -0.75    0.      2.75    1.   ]
 [  0.125   0.     -0.125   0.   ]]

Проверка решения (Ax - b):
[3.55271368e-15 0.00000000e+00 8.88178420e-16 0.00000000e+00]

Норма невязки: 3.66e-15
Невязка минимальна, решение корректно!


## 1.2 Метод прогонки

In [22]:
import os

import numpy as np

# Чтение входной матрицы
with open("input/1_2.txt", "r") as f:
    A_ext = np.array(
        [list(map(float, line.strip().split())) for line in f if line.strip()]
    )

# Создание папки output, если нет
os.makedirs("output", exist_ok=True)

n = A_ext.shape[0]

# Матрица коэффициентов (первые n столбцов)
A = A_ext[:, :-1]

# Вектор правой части
d = A_ext[:, -1]

output = ""
print("Матрица коэффициентов A:")
print(A)
output += "Матрица коэффициентов A:\n"
output += f"{A}\n\n"

print("\nВектор правой части d:")
print(d)
output += "Вектор правой части d:\n"
output += f"{d}\n"

# Извлечение диагоналей
a = np.zeros(n)
b = np.zeros(n)
c = np.zeros(n)

for i in range(n):
    for j in range(n):
        if i == j:
            b[i] = A[i, j]
        elif i == j + 1:
            a[i] = A[i, j]
        elif i == j - 1:
            c[i] = A[i, j]
        elif A[i, j] != 0:
            raise ValueError("Матрица не трёхдиагональная!")

# Прямой ход
P = np.zeros(n)
Q = np.zeros(n)
P[0] = -c[0] / b[0]
Q[0] = d[0] / b[0]

for i in range(1, n):
    denominator = b[i] + a[i] * P[i - 1]
    if i != n - 1:
        P[i] = -c[i] / denominator
    Q[i] = (d[i] - a[i] * Q[i - 1]) / denominator

# Обратный ход
x = np.zeros(n)
x[-1] = Q[-1]

for i in range(n - 2, -1, -1):
    x[i] = P[i] * x[i + 1] + Q[i]

print("\nРешение системы (вектор x):")
print(x)
output += "Решение системы (вектор x):\n"
output += f"{x}\n\n"

# Проверка решения
r = A @ x - d  # Вектор невязки
residual_norm = np.linalg.norm(r)

print("\nПроверка решения (Ax - d):")
print(r)
output += "Проверка решения (Ax - d):\n"
output += f"{r}\n\n"

print(f"\nНорма невязки: {residual_norm:.2e}")
output += f"Норма невязки: {residual_norm:.2e}\n"

if residual_norm < 1e-8:
    print("Невязка минимальна, решение корректно!")
    output += "Невязка минимальна, решение корректно!\n"
else:
    print("Обнаружена значительная невязка!")
    output += "Обнаружена значительная невязка!\n"

# Запись вывода в файл
with open("output/1_2.txt", "w", encoding="utf-8") as f:
    f.write(output)

Матрица коэффициентов A:
[[-11.  -9.   0.   0.   0.]
 [  5. -15.  -2.   0.   0.]
 [  0.  -8.  11.  -3.   0.]
 [  0.   0.   6. -15.   4.]
 [  0.   0.   0.   3.   6.]]

Вектор правой части d:
[-122.  -48.  -14.  -50.   42.]

Решение системы (вектор x):
[7. 5. 4. 6. 4.]

Проверка решения (Ax - d):
[ 0.00000000e+00 -1.42108547e-14  0.00000000e+00  0.00000000e+00
  7.10542736e-15]

Норма невязки: 1.59e-14
Невязка минимальна, решение корректно!


## 1.3 Метод простых итераций и метод Зейделя

In [23]:
import os

import numpy as np

# Чтение входных данных
with open("input/1_3.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

epsilon = float(lines[0])
A_ext = np.array([list(map(float, line.split())) for line in lines[1:]])

# Создание папки output, если нет
os.makedirs("output", exist_ok=True)

A = A_ext[:, :-1]
b = A_ext[:, -1]

output = ""
print("Матрица A:")
print(A)
output += "Матрица A:\n"
output += f"{A}\n\n"

print("\nВектор b:")
print(b)
output += "Вектор b:\n"
output += f"{b}\n\n"

print(f"\nТочность вычислений: {epsilon:e}")
output += f"Точность вычислений: {epsilon:e}\n\n"

n = len(b)

# Метод простых итераций
x = np.zeros(n)

for iteration in range(1000):
    x_new = np.zeros(n)
    for i in range(n):
        x_new[i] = (
            b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i + 1 :], x[i + 1 :])
        ) / A[i, i]

    if np.linalg.norm(x_new - x) < epsilon:
        iter_x = x_new
        iter_amount = iteration + 1
        break
    x = x_new
else:
    iter_x = x
    iter_amount = 1000

residual_iter = A @ iter_x - b
max_residual_iter = np.max(np.abs(residual_iter))

print("\nМетод простых итераций:")
print(f"Количество итераций: {iter_amount}")
print("Решение:")
print(iter_x)
print(f"Максимальная невязка: {max_residual_iter:.2e}")

output += "Метод простых итераций:\n"
output += f"Количество итераций: {iter_amount}\n"
output += "Решение:\n"
output += f"{iter_x}\n"
output += f"Максимальная невязка: {max_residual_iter:.2e}\n\n"

# Метод Зейделя
x = np.zeros(n)

for iteration in range(1000):
    x_prev = x.copy()
    for i in range(n):
        x[i] = (
            b[i] - np.dot(A[i, :i], x[:i]) - np.dot(A[i, i + 1 :], x_prev[i + 1 :])
        ) / A[i, i]

    if np.linalg.norm(x - x_prev) < epsilon:
        zeidel_x = x
        zeidel_amount = iteration + 1
        break
else:
    zeidel_x = x
    zeidel_amount = 1000

residual_zeidel = A @ zeidel_x - b
max_residual_zeidel = np.max(np.abs(residual_zeidel))

print("\nМетод Зейделя:")
print(f"Количество итераций: {zeidel_amount}")
print("Решение:")
print(zeidel_x)
print(f"Максимальная невязка: {max_residual_zeidel:.2e}")

output += "Метод Зейделя:\n"
output += f"Количество итераций: {zeidel_amount}\n"
output += "Решение:\n"
output += f"{zeidel_x}\n"
output += f"Максимальная невязка: {max_residual_zeidel:.2e}\n"

# Запись вывода в файл
with open("output/1_3.txt", "w", encoding="utf-8") as f:
    f.write(output)

Матрица A:
[[ 19.  -4.  -9.  -1.]
 [ -2.  20.  -2.  -7.]
 [  6.  -5. -25.   9.]
 [  0.  -3.  -9.  12.]]

Вектор b:
[100.  -5.  34.  69.]

Точность вычислений: 1.000000e-06

Метод простых итераций:
Количество итераций: 50
Решение:
[7.99999986 4.00000011 3.00000019 8.99999976]
Максимальная невязка: 8.34e-06

Метод Зейделя:
Количество итераций: 22
Решение:
[7.99999963 3.99999974 2.99999978 8.99999977]
Максимальная невязка: 3.82e-06


## 1.4 Метод вращений

In [24]:
import os
import numpy as np
import plotly.graph_objects as go

# Чтение входных данных
with open("input/1_4.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

epsilon = float(lines[0])
a = np.array([list(map(float, line.split())) for line in lines[1:]])

# Создание папки output, если нет
os.makedirs("output", exist_ok=True)

output = ""
print("Исходная матрица A:")
print(a)
output += "Исходная матрица A:\n"
output += f"{a}\n\n"

print(f"\nТочность вычислений: {epsilon:e}")
output += f"Точность вычислений: {epsilon:e}\n\n"

n = a.shape[0]
eigenvectors = np.eye(n)
iterations = 0
max_iterations = 1000  # Максимальное число итераций

# Истинные собственные значения для контроля погрешности
true_eigenvalues = np.linalg.eigvals(a)
true_eigenvalues.sort()

# Для графика
errors = []
iteration_counts = []

# Метод вращений с ограничением итераций
for _ in range(max_iterations):
    upper_tri = np.triu(a, k=1)
    p, q = np.unravel_index(np.argmax(np.abs(upper_tri)), a.shape)
    max_val = abs(a[p, q])

    current_eigenvalues = np.diag(a).copy()
    current_eigenvalues.sort()

    error = np.sqrt(np.mean((current_eigenvalues - true_eigenvalues) ** 2))
    errors.append(error)
    iteration_counts.append(iterations)

    if max_val < epsilon * np.max(np.abs(np.diag(a))):
        break

    theta = 0.5 * np.arctan2(2 * a[p, q], a[p, p] - a[q, q])

    rotation = np.eye(n)
    c, s = np.cos(theta), np.sin(theta)
    rotation[p, p] = c
    rotation[q, q] = c
    rotation[p, q] = -s
    rotation[q, p] = s

    a = rotation.T @ a @ rotation
    eigenvectors = eigenvectors @ rotation
    iterations += 1

else:
    print(f"\nДостигнуто максимальное количество итераций ({max_iterations}).")
    output += f"Достигнуто максимальное количество итераций ({max_iterations}).\n\n"

# Финальные результаты
eigenvalues = np.diag(a)

print("\nСобственные значения:")
print(eigenvalues)
output += "Собственные значения:\n"
output += f"{eigenvalues}\n\n"

print("\nСобственные векторы (столбцы матрицы):")
print(eigenvectors)
output += "Собственные векторы (столбцы матрицы):\n"
output += f"{eigenvectors}\n\n"

print(f"\nКоличество итераций: {iterations}")
output += f"Количество итераций: {iterations}\n"

# Запись вывода в файл
with open("output/1_4.txt", "w", encoding="utf-8") as f:
    f.write(output)

# Построение графика через Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=iteration_counts, y=errors, mode="lines+markers"))

fig.update_layout(
    title="Зависимость погрешности от числа итераций",
    xaxis_title="Количество итераций",
    yaxis_title="Среднеквадратичная погрешность",
    template="plotly_white",
)

fig.show()

Исходная матрица A:
[[-7.  4.  5.]
 [ 4. -6. -9.]
 [ 5. -9. -8.]]

Точность вычислений: 1.000000e-08

Собственные значения:
[  2.07377213  -3.7106485  -19.36312363]

Собственные векторы (столбцы матрицы):
[[ 0.05184815  0.88666192 -0.45950235]
 [-0.73180676  0.34682369  0.58666191]
 [ 0.67953707  0.30584959  0.66684736]]

Количество итераций: 6


## 1.5 Метод QR-разложения матриц

In [25]:
import os

import numpy as np
from scipy.linalg import hessenberg, qr

# Чтение входных данных
with open("input/1_5.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

epsilon = float(lines[0])
A_original = np.array([list(map(float, line.split())) for line in lines[1:]])

# Создание папки output, если нет
os.makedirs("output", exist_ok=True)

output = ""
print("Исходная матрица A:")
print(A_original)
output += "Исходная матрица A:\n"
output += f"{A_original}\n\n"

print(f"\nТочность вычислений: {epsilon:e}")
output += f"Точность вычислений: {epsilon:e}\n\n"

# Подготовка
A = np.array(A_original, dtype=complex)  # Для поддержки комплексных чисел
n = A.shape[0]
assert A.shape[1] == n, "Матрица должна быть квадратной"

# Приведение к хессенберговой форме
A = hessenberg(A)

print("\nХессенбергова форма H (перед началом итераций):")
print(np.round(A, 4))
output += "Хессенбергова форма H (перед началом итераций):\n"
output += f"{np.round(A, 4)}\n\n"

# QR-алгоритм
max_iter = 1000
for k in range(max_iter):
    if n >= 2:
        T = A[-2:, -2:]
        eigenvals = np.linalg.eigvals(T)
        last_diag = A[-1, -1]
        shift = eigenvals[np.argmin(np.abs(eigenvals - last_diag))]
    else:
        shift = A[-1, -1]

    Q, R = qr(A - shift * np.eye(n))
    A = R @ Q + shift * np.eye(n)

    off_diag_norm = np.linalg.norm(np.tril(A, -1))

    if off_diag_norm < epsilon:
        print(f"\nДостигнута сходимость на {k+1}-й итерации (epsilon = {epsilon:e})")
        output += (
            f"Достигнута сходимость на {k+1}-й итерации (epsilon = {epsilon:e})\n\n"
        )
        break
else:
    print(f"\nНе достигнута сходимость за {max_iter} итераций.")
    output += f"Не достигнута сходимость за {max_iter} итераций.\n\n"

# После завершения итераций
eigenvalues = np.diag(A)

print("\nСобственные значения матрицы:")
print(np.round(eigenvalues, 6))
output += "Собственные значения матрицы:\n"
output += f"{np.round(eigenvalues, 6)}\n"

# Запись вывода в файл
with open("output/1_5.txt", "w", encoding="utf-8") as f:
    f.write(output)

Исходная матрица A:
[[ 3. -7. -1.]
 [-9. -8.  7.]
 [ 5.  2.  2.]]

Точность вычислений: 1.000000e-06

Хессенбергова форма H (перед началом итераций):
[[ 3.    +0.j  5.6335+0.j -4.2737+0.j]
 [10.2956+0.j -9.4623+0.j -0.6321+0.j]
 [ 0.    +0.j  4.3679+0.j  3.4623+0.j]]

Достигнута сходимость на 14-й итерации (epsilon = 1.000000e-06)

Собственные значения матрицы:
[-13.501042+0.j         5.250521+2.864794j   5.250521-2.864794j]


# Лабораторная работа 2. Метод решения нелинейных уравнений

## 2.1 Методы простой итерации и Ньютона для уравнения

In [26]:
import math
import os

import numpy as np
import plotly.graph_objects as go

# Создание папки output, если нет
os.makedirs("output", exist_ok=True)

output = ""


# Заданные функции
def f(x):
    return 2**x - x**2 - 0.5


def df(x):
    return 2**x * math.log(2) - 2 * x


def g(x):
    return math.sqrt(2**x - 0.5)


# Параметры
epsilon = 1e-6
max_iterations = 1000
x0 = 1.5  # начальное приближение

# Вывод начального уравнения
print("Начальное уравнение:")
print("f(x) = 2^x - x^2 - 0.5")
output += "Начальное уравнение:\n"
output += "f(x) = 2^x - x^2 - 0.5\n\n"

# Построение графика функции
x_vals = np.linspace(-1, 3, 400)
y_vals = [f(x) for x in x_vals]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode="lines", name="f(x)"))
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=[0] * len(x_vals),
        mode="lines",
        name="y = 0",
        line=dict(dash="dash"),
    )
)

fig.update_layout(
    title="График функции для определения начального приближения",
    xaxis_title="x",
    yaxis_title="f(x)",
    template="plotly_white",
)

fig.show()

# Метод Ньютона
print("\nМетод Ньютона:")
print(f"Начальное приближение: x0 = {x0}")
output += "Метод Ньютона:\n"
output += f"Начальное приближение: x0 = {x0}\n\n"

x = x0
for iteration in range(1, max_iterations + 1):
    fx = f(x)
    dfx = df(x)
    x_new = x - fx / dfx

    print(f"Итерация {iteration}: x = {x:.6f}, f(x) = {fx:.6f}, f'(x) = {dfx:.6f}")
    output += f"Итерация {iteration}: x = {x:.6f}, f(x) = {fx:.6f}, f'(x) = {dfx:.6f}\n"

    if abs(x_new - x) < epsilon:
        break
    x = x_new

print(
    f"\nПоложительный корень (метод Ньютона): x ≈ {x_new:.6f} после {iteration} итераций при ε = {epsilon}"
)
output += (
    f"\nПоложительный корень (метод Ньютона): x ≈ {x_new:.6f} "
    f"после {iteration} итераций при ε = {epsilon}\n\n"
)

# Метод простой итерации
print("\nМетод простой итерации:")
print(f"Начальное приближение: x0 = {x0}")
output += "Метод простой итерации:\n"
output += f"Начальное приближение: x0 = {x0}\n\n"

x = x0
for iteration in range(1, max_iterations + 1):
    x_new = g(x)

    print(f"Итерация {iteration}: x = {x:.6f}")
    output += f"Итерация {iteration}: x = {x:.6f}\n"

    if abs(x_new - x) < epsilon:
        break
    x = x_new

print(
    f"\nПоложительный корень (метод простой итерации): x ≈ {x_new:.6f} после {iteration} итераций при ε = {epsilon}"
)
output += (
    f"\nПоложительный корень (метод простой итерации): x ≈ {x_new:.6f} "
    f"после {iteration} итераций при ε = {epsilon}\n"
)

# Запись вывода в файл
with open("output/2_1.txt", "w", encoding="utf-8") as f:
    f.write(output)

Начальное уравнение:
f(x) = 2^x - x^2 - 0.5



Метод Ньютона:
Начальное приближение: x0 = 1.5
Итерация 1: x = 1.500000, f(x) = 0.078427, f'(x) = -1.039484
Итерация 2: x = 1.575448, f(x) = -0.001756, f'(x) = -1.085123
Итерация 3: x = 1.573830, f(x) = -0.000001, f'(x) = -1.084202

Положительный корень (метод Ньютона): x ≈ 1.573829 после 3 итераций при ε = 1e-06

Метод простой итерации:
Начальное приближение: x0 = 1.5
Итерация 1: x = 1.500000
Итерация 2: x = 1.525918
Итерация 3: x = 1.542628
Итерация 4: x = 1.553463
Итерация 5: x = 1.560515
Итерация 6: x = 1.565117
Итерация 7: x = 1.568125
Итерация 8: x = 1.570092
Итерация 9: x = 1.571381
Итерация 10: x = 1.572225
Итерация 11: x = 1.572777
Итерация 12: x = 1.573140
Итерация 13: x = 1.573377
Итерация 14: x = 1.573533
Итерация 15: x = 1.573635
Итерация 16: x = 1.573702
Итерация 17: x = 1.573745
Итерация 18: x = 1.573774
Итерация 19: x = 1.573793
Итерация 20: x = 1.573805
Итерация 21: x = 1.573814
Итерация 22: x = 1.573819
Итерация 23: x = 1.573822
Итерация 24: x = 1.573825
Итерация 25:

## 2.2 Методы простой итерации и Ньютона для системы уравнений

In [27]:
import math
import os

import numpy as np
import plotly.graph_objects as go

# Создание папки output, если нет
os.makedirs("output", exist_ok=True)

output = ""

# Параметры
epsilon = 1e-6
max_iterations = 1000


# Функции системы
def f1(x1, x2):
    return (x1**2 + 4) * x2 - 8


def f2(x1, x2):
    return (x1 - 1) ** 2 + (x2 - 1) ** 2 - 4


# Частные производные
def df1_dx1(x1, x2):
    return 2 * x1 * x2


def df1_dx2(x1, x2):
    return x1**2 + 4


def df2_dx1(x1, x2):
    return 2 * (x1 - 1)


def df2_dx2(x1, x2):
    return 2 * (x2 - 1)


# Функции простой итерации
def phi1(x2):
    return 1 + math.sqrt(4 - (x2 - 1) ** 2)


def phi2(x1):
    return 8 / (x1**2 + 4)


# Вывод начальной системы
print("Исходная система уравнений:")
print("f1(x1, x2) = (x1^2 + 4) * x2 - 8")
print("f2(x1, x2) = (x1 - 1)^2 + (x2 - 1)^2 - 4")

output += "Исходная система уравнений:\n"
output += "f1(x1, x2) = (x1^2 + 4) * x2 - 8\n"
output += "f2(x1, x2) = (x1 - 1)^2 + (x2 - 1)^2 - 4\n\n"

print(f"\nТочность вычислений: {epsilon:e}")
output += f"Точность вычислений: {epsilon:e}\n\n"

# Построение графика системы
x1_vals = np.linspace(-2, 4, 400)
x2_vals = np.linspace(-2, 4, 400)
X1, X2 = np.meshgrid(x1_vals, x2_vals)

F1 = (X1**2 + 4) * X2 - 8
F2 = (X1 - 1) ** 2 + (X2 - 1) ** 2 - 4

fig = go.Figure()

fig.add_trace(
    go.Contour(
        z=F1,
        x=x1_vals,
        y=x2_vals,
        colorscale="Blues",
        showscale=False,
        contours=dict(start=0, end=0, size=1, coloring="lines"),
        line=dict(width=3),
        name="f1(x1, x2) = 0",
    )
)

fig.add_trace(
    go.Contour(
        z=F2,
        x=x1_vals,
        y=x2_vals,
        colorscale="Reds",
        showscale=False,
        contours=dict(start=0, end=0, size=1, coloring="lines"),
        line=dict(width=3),
        name="f2(x1, x2) = 0",
    )
)

fig.update_layout(
    title="Графики функций системы",
    xaxis_title="x1",
    yaxis_title="x2",
    template="plotly_white",
)

fig.show()

# Начальное приближение (выбираем положительные)
x1, x2 = 2.5, 0.7
print(f"\nНачальное приближение: x1 = {x1}, x2 = {x2}")
output += f"Начальное приближение: x1 = {x1}, x2 = {x2}\n\n"

# Метод Ньютона
errors_newton = []
print("\nМетод Ньютона:")
output += "Метод Ньютона:\n"

for iteration in range(1, max_iterations + 1):
    f1_val = f1(x1, x2)
    f2_val = f2(x1, x2)

    J = np.array(
        [[df1_dx1(x1, x2), df1_dx2(x1, x2)], [df2_dx1(x1, x2), df2_dx2(x1, x2)]]
    )
    F = np.array([-f1_val, -f2_val])

    delta = np.linalg.lstsq(J, F, rcond=None)[0]

    x1_new = x1 + delta[0]
    x2_new = x2 + delta[1]

    norm = np.linalg.norm(delta)
    errors_newton.append(norm)

    print(f"Итерация {iteration}: x1 = {x1:.6f}, x2 = {x2:.6f}, ||Δx|| = {norm:.2e}")
    output += (
        f"Итерация {iteration}: x1 = {x1:.6f}, x2 = {x2:.6f}, ||Δx|| = {norm:.2e}\n"
    )

    if norm < epsilon:
        break

    x1, x2 = x1_new, x2_new

print(
    f"\nПоложительный корень (метод Ньютона): x1 ≈ {x1_new:.6f}, x2 ≈ {x2_new:.6f} после {iteration} итераций."
)
output += (
    f"\nПоложительный корень (метод Ньютона): x1 ≈ {x1_new:.6f}, x2 ≈ {x2_new:.6f} "
    f"после {iteration} итераций.\n\n"
)

# Метод простой итерации
errors_simple = []
x1, x2 = 2.0, 1.5
print("\nМетод простой итерации:")
output += "Метод простой итерации:\n"

for iteration in range(1, max_iterations + 1):
    x1_new = phi1(x2)
    x2_new = phi2(x1_new)

    error = max(abs(x1_new - x1), abs(x2_new - x2))
    errors_simple.append(error)

    print(f"Итерация {iteration}: x1 = {x1:.6f}, x2 = {x2:.6f}, error = {error:.2e}")
    output += (
        f"Итерация {iteration}: x1 = {x1:.6f}, x2 = {x2:.6f}, error = {error:.2e}\n"
    )

    if error < epsilon:
        break

    x1, x2 = x1_new, x2_new

print(
    f"\nПоложительный корень (метод простой итерации): x1 ≈ {x1_new:.6f}, x2 ≈ {x2_new:.6f} после {iteration} итераций."
)
output += (
    f"\nПоложительный корень (метод простой итерации): x1 ≈ {x1_new:.6f}, x2 ≈ {x2_new:.6f} "
    f"после {iteration} итераций.\n"
)

# Запись вывода в файл
with open("output/2_2.txt", "w", encoding="utf-8") as f:
    f.write(output)

Исходная система уравнений:
f1(x1, x2) = (x1^2 + 4) * x2 - 8
f2(x1, x2) = (x1 - 1)^2 + (x2 - 1)^2 - 4

Точность вычислений: 1.000000e-06



Начальное приближение: x1 = 2.5, x2 = 0.7

Метод Ньютона:
Итерация 1: x1 = 2.500000, x2 = 0.700000, ||Δx|| = 5.43e-01
Итерация 2: x1 = 3.033029, x2 = 0.598478, ||Δx|| = 7.21e-02
Итерация 3: x1 = 2.965776, x2 = 0.624592, ||Δx|| = 1.48e-03
Итерация 4: x1 = 2.964632, x2 = 0.625535, ||Δx|| = 7.25e-07

Положительный корень (метод Ньютона): x1 ≈ 2.964631, x2 ≈ 0.625536 после 4 итераций.

Метод простой итерации:
Итерация 1: x1 = 2.000000, x2 = 1.500000, error = 9.36e-01
Итерация 2: x1 = 2.936492, x2 = 0.633765, error = 2.97e-02
Итерация 3: x1 = 2.966182, x2 = 0.625086, error = 1.64e-03
Итерация 4: x1 = 2.964546, x2 = 0.625561, error = 9.05e-05
Итерация 5: x1 = 2.964636, x2 = 0.625534, error = 5.00e-06
Итерация 6: x1 = 2.964631, x2 = 0.625536, error = 2.76e-07

Положительный корень (метод простой итерации): x1 ≈ 2.964631, x2 ≈ 0.625536 после 6 итераций.


# Лабораторная работа 3. Методы приближения функций. Численное дифференцирование и интегрирование

## 3.1 Интерполяционные члены Лагранжа и Ньютона

In [28]:
import os

import numpy as np
import plotly.graph_objects as go

# --- Создание папки для вывода ---
os.makedirs("output", exist_ok=True)


# --- Функции для интерполяции ---
def lagrange_interpolation(x, X, Y):
    total = 0
    n = len(X)
    for i in range(n):
        term = Y[i]
        for j in range(n):
            if j != i:
                term *= (x - X[j]) / (X[i] - X[j])
        total += term
    return total


def newton_coefficients(X, Y):
    n = len(X)
    coef = np.copy(Y)
    for j in range(1, n):
        coef[j:n] = (coef[j:n] - coef[j - 1 : n - 1]) / (X[j:n] - X[0 : n - j])
    return coef


def newton_interpolation(x, X, coef):
    n = len(X)
    result = coef[-1]
    for k in range(2, n + 1):
        result = coef[-k] + (x - X[-k]) * result
    return result


# --- Начало работы ---
output = ""

# --- Данные для двух вариантов ---
variants = {
    "a": np.array([0.1 * np.pi, 0.2 * np.pi, 0.3 * np.pi, 0.4 * np.pi]),
    "б": np.array([0.1 * np.pi, np.pi / 6, 0.3 * np.pi, 0.4 * np.pi]),
}

X_star = np.pi / 4  # Точка для интерполяции

# --- Основной цикл по вариантам ---
for label, X in variants.items():
    Y = np.sin(X)

    output += f"Вариант {label}:\n"
    print(f"Вариант {label}:")

    # Таблица значений
    output += "Таблица значений:\n"
    output += f"X = {X}\n"
    output += f"Y = {Y}\n"

    print("Таблица значений:")
    print(f"X = {X}")
    print(f"Y = {Y}")

    # Лагранж
    P_L = lagrange_interpolation(X_star, X, Y)
    output += f"\nЗначение многочлена Лагранжа в точке X* = {X_star}: {P_L}\n"
    print(f"\nЗначение многочлена Лагранжа в точке X* = {P_L}")

    # Ньютон
    coef_newton = newton_coefficients(X, Y)
    P_N = newton_interpolation(X_star, X, coef_newton)
    output += f"Значение многочлена Ньютона в точке X* = {X_star}: {P_N}\n"
    print(f"Значение многочлена Ньютона в точке X* = {P_N}")

    # Истинное значение и погрешности
    true_value = np.sin(X_star)
    error_L = abs(true_value - P_L)
    error_N = abs(true_value - P_N)

    output += f"\nИстинное значение sin(X*) = {true_value}\n"
    output += f"Погрешность Лагранжа = {error_L}\n"
    output += f"Погрешность Ньютона = {error_N}\n"
    output += "\n" + "-" * 50 + "\n\n"

    print(f"\nИстинное значение sin(X*) = {true_value}")
    print(f"Погрешность Лагранжа = {error_L}")
    print(f"Погрешность Ньютона = {error_N}")
    print("-" * 50)

    # --- Построение графика ---
    x_plot = np.linspace(min(X) - 0.1, max(X) + 0.1, 300)
    y_true = np.sin(x_plot)
    y_lagrange = np.array([lagrange_interpolation(xi, X, Y) for xi in x_plot])
    y_newton = np.array([newton_interpolation(xi, X, coef_newton) for xi in x_plot])

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=x_plot, y=y_true, mode="lines", name="sin(x)", line=dict(dash="dot")
        )
    )
    fig.add_trace(go.Scatter(x=x_plot, y=y_lagrange, mode="lines", name="Лагранж"))
    fig.add_trace(go.Scatter(x=x_plot, y=y_newton, mode="lines", name="Ньютон"))
    fig.add_trace(
        go.Scatter(
            x=X, y=Y, mode="markers", name="Узлы интерполяции", marker=dict(size=10)
        )
    )

    fig.update_layout(
        title=f"Интерполяция для варианта {label}",
        xaxis_title="x",
        yaxis_title="y",
        legend_title="Легенда",
        width=800,
        height=500,
    )

    fig.show()

# --- Сохранение текстового вывода ---
with open("output/3_1.txt", "w", encoding="utf-8") as f:
    f.write(output)

Вариант a:
Таблица значений:
X = [0.31415927 0.62831853 0.9424778  1.25663706]
Y = [0.30901699 0.58778525 0.80901699 0.95105652]

Значение многочлена Лагранжа в точке X* = 0.7069466693335428
Значение многочлена Ньютона в точке X* = 0.7069466693335428

Истинное значение sin(X*) = 0.7071067811865476
Погрешность Лагранжа = 0.00016011185300479625
Погрешность Ньютона = 0.00016011185300479625
--------------------------------------------------


Вариант б:
Таблица значений:
X = [0.31415927 0.52359878 0.9424778  1.25663706]
Y = [0.30901699 0.5        0.80901699 0.95105652]

Значение многочлена Лагранжа в точке X* = 0.7068457655581606
Значение многочлена Ньютона в точке X* = 0.7068457655581606

Истинное значение sin(X*) = 0.7071067811865476
Погрешность Лагранжа = 0.00026101562838698467
Погрешность Ньютона = 0.00026101562838698467
--------------------------------------------------


## 3.2 Кубический сплайн

In [29]:
import os

import numpy as np
import plotly.graph_objects as go

# --- Создание папки ---
os.makedirs("output", exist_ok=True)

# --- Исходные данные ---
output = ""

# Таблица значений
x = np.array([0.0, 1.0, 2.0, 3.0, 4.0])
f = np.array([0.0, 0.5, 0.86603, 1.0, 0.86603])

n = len(x) - 1  # количество интервалов
h = np.diff(x)  # шаги между узлами

print("Таблица значений:")
print(f"x = {x}")
print(f"f = {f}\n")

output += "Таблица значений:\n"
output += f"x = {x}\n"
output += f"f = {f}\n\n"

# --- Составляем СЛАУ для коэффициентов c (вторые производные) ---
A = np.zeros((n + 1, n + 1))
rhs = np.zeros(n + 1)

# Условия натурального сплайна
A[0, 0] = 1
A[n, n] = 1

for i in range(1, n):
    A[i, i - 1] = h[i - 1]
    A[i, i] = 2 * (h[i - 1] + h[i])
    A[i, i + 1] = h[i]
    rhs[i] = 3 * ((f[i + 1] - f[i]) / h[i] - (f[i] - f[i - 1]) / h[i - 1])

# Решаем СЛАУ
c = np.linalg.solve(A, rhs)

# Находим коэффициенты b и d
b = np.zeros(n)
d = np.zeros(n)
a = f[:-1]

for i in range(n):
    b[i] = (f[i + 1] - f[i]) / h[i] - h[i] * (2 * c[i] + c[i + 1]) / 3
    d[i] = (c[i + 1] - c[i]) / (3 * h[i])

# --- Вывод коэффициентов ---
print("Коэффициенты сплайна на каждом интервале [xi, xi+1]:")
output += "Коэффициенты сплайна на каждом интервале [xi, xi+1]:\n"

for i in range(n):
    print(f"[{x[i]}, {x[i+1]}]: a = {a[i]}, b = {b[i]}, c = {c[i]}, d = {d[i]}")
    output += f"[{x[i]}, {x[i+1]}]: a = {a[i]}, b = {b[i]}, c = {c[i]}, d = {d[i]}\n"

# --- Вычисление значения в точке x* = 1.5 ---
x_star = 1.5

# Определяем, на каком интервале находится x_star
for i in range(n):
    if x[i] <= x_star <= x[i + 1]:
        S_star = (
            a[i]
            + b[i] * (x_star - x[i])
            + c[i] * (x_star - x[i]) ** 2
            + d[i] * (x_star - x[i]) ** 3
        )
        break

print(f"\nЗначение сплайна в точке x* = {x_star}: {S_star}")
output += f"\nЗначение сплайна в точке x* = {x_star}: {S_star}\n"

# --- Построение графика сплайна ---
x_plot = np.linspace(x[0], x[-1], 500)
y_plot = []

for xp in x_plot:
    for i in range(n):
        if x[i] <= xp <= x[i + 1]:
            S = (
                a[i]
                + b[i] * (xp - x[i])
                + c[i] * (xp - x[i]) ** 2
                + d[i] * (xp - x[i]) ** 3
            )
            y_plot.append(S)
            break

# Создание графика
fig = go.Figure()

# График сплайна
fig.add_trace(go.Scatter(x=x_plot, y=y_plot, mode="lines", name="Кубический сплайн"))

# Исходные точки
fig.add_trace(
    go.Scatter(
        x=x,
        y=f,
        mode="markers+text",
        name="Узлы",
        text=[f"{fi:.2f}" for fi in f],
        textposition="top center",
    )
)

# Отдельно точка x* = 1.5
fig.add_trace(
    go.Scatter(
        x=[x_star],
        y=[S_star],
        mode="markers",
        name="Точка x*",
        marker=dict(size=10, color="red"),
    )
)

fig.update_layout(
    title="Кубический сплайн для таблицы значений",
    xaxis_title="x",
    yaxis_title="y",
    legend_title="Легенда",
)

fig.show()

# --- Сохранение в файл ---
with open("output/3_2.txt", "w", encoding="utf-8") as file_out:
    file_out.write(output)

Таблица значений:
x = [0. 1. 2. 3. 4.]
f = [0.      0.5     0.86603 1.      0.86603]

Коэффициенты сплайна на каждом интервале [xi, xi+1]:
[0.0, 1.0]: a = 0.0, b = 0.52409375, c = 0.0, d = -0.024093750000000014
[1.0, 2.0]: a = 0.5, b = 0.45181249999999995, c = -0.07228125000000005, d = -0.01350124999999996
[2.0, 3.0]: a = 0.86603, b = 0.26674624999999996, c = -0.11278499999999993, d = -0.019991250000000047
[3.0, 4.0]: a = 1.0, b = -0.01879749999999998, c = -0.17275875000000007, d = 0.057586250000000026

Значение сплайна в точке x* = 1.5: 0.70614828125


## 3.3 Приближающие многочлены

In [30]:
import os

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

# --- Настройка рендеринга plotly ---
pio.renderers.default = "notebook_connected"

# --- Создание папки ---
os.makedirs("output", exist_ok=True)

# --- Исходные данные ---
output = ""

x = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0])
y = np.array([-0.5, 0.0, 0.5, 0.86603, 1.0, 0.86603])

print("Таблица значений:")
print(f"x = {x}")
print(f"y = {y}\n")

output += "Таблица значений:\n"
output += f"x = {x}\n"
output += f"y = {y}\n\n"


# --- Функция для МНК ---
def least_squares(x, y, degree):
    X = np.vander(x, degree + 1, increasing=True)
    coeffs = np.linalg.lstsq(X, y, rcond=None)[0]
    return coeffs


# --- Аппроксимация первой степени (прямая) ---
coeffs_deg1 = least_squares(x, y, degree=1)

print("Коэффициенты приближающего многочлена 1-й степени:")
print(coeffs_deg1)
output += "Коэффициенты приближающего многочлена 1-й степени:\n"
output += f"{coeffs_deg1}\n\n"

# --- Аппроксимация второй степени (парабола) ---
coeffs_deg2 = least_squares(x, y, degree=2)

print("\nКоэффициенты приближающего многочлена 2-й степени:")
print(coeffs_deg2)
output += "Коэффициенты приближающего многочлена 2-й степени:\n"
output += f"{coeffs_deg2}\n\n"


# --- Вычисление суммы квадратов ошибок ---
def compute_sse(x, y, coeffs):
    X = np.vander(x, len(coeffs), increasing=True)
    y_pred = X @ coeffs
    return np.sum((y - y_pred) ** 2)


sse_deg1 = compute_sse(x, y, coeffs_deg1)
sse_deg2 = compute_sse(x, y, coeffs_deg2)

print(f"\nСумма квадратов ошибок для 1-й степени: {sse_deg1}")
print(f"Сумма квадратов ошибок для 2-й степени: {sse_deg2}")

output += f"Сумма квадратов ошибок для 1-й степени: {sse_deg1}\n"
output += f"Сумма квадратов ошибок для 2-й степени: {sse_deg2}\n"

# --- Сохранение текстового вывода ---
with open("output/3_3.txt", "w", encoding="utf-8") as file_out:
    file_out.write(output)

# --- Построение графика ---
x_plot = np.linspace(min(x), max(x), 500)

# Значения аппроксимирующих многочленов
y_plot_deg1 = coeffs_deg1[0] + coeffs_deg1[1] * x_plot
y_plot_deg2 = coeffs_deg2[0] + coeffs_deg2[1] * x_plot + coeffs_deg2[2] * x_plot**2

# Создание графика
fig = go.Figure()

# Табличные точки
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode="markers+text",
        name="Табличные точки",
        text=[f"{yi:.2f}" for yi in y],
        textposition="top center",
    )
)

# Аппроксимирующий многочлен 1-й степени
fig.add_trace(
    go.Scatter(x=x_plot, y=y_plot_deg1, mode="lines", name="Аппроксимация 1-й степени")
)

# Аппроксимирующий многочлен 2-й степени
fig.add_trace(
    go.Scatter(x=x_plot, y=y_plot_deg2, mode="lines", name="Аппроксимация 2-й степени")
)

fig.update_layout(
    title="Аппроксимация табличной функции МНК",
    xaxis_title="x",
    yaxis_title="y",
    legend_title="Легенда",
)

fig.show()

Таблица значений:
x = [-1.  0.  1.  2.  3.  4.]
y = [-0.5      0.       0.5      0.86603  1.       0.86603]

Коэффициенты приближающего многочлена 1-й степени:
[0.01836419 0.29131943]

Коэффициенты приближающего многочлена 2-й степени:
[ 0.0735305   0.53956782 -0.08274946]

Сумма квадратов ошибок для 1-й степени: 0.2708179489276191
Сумма квадратов ошибок для 2-й степени: 0.015178925583571454


## 3.4 Первая и вторая производные от таблично заданной функции

In [31]:
import os

import numpy as np

# --- Создание папки для вывода ---
os.makedirs("output", exist_ok=True)

# --- Исходные данные ---
output = ""

x = np.array([-1.0, 0.0, 1.0, 2.0, 3.0])
y = np.array([-0.5, 0.0, 0.5, 0.86603, 1.0])

x_star = 1.0  # Точка, в которой нужно найти производные

print("Таблица значений:")
print(f"x = {x}")
print(f"y = {y}\n")

output += "Таблица значений:\n"
output += f"x = {x}\n"
output += f"y = {y}\n\n"

# --- Поиск индекса точки x* ---
index_star = np.where(x == x_star)[0][0]

# Проверка на возможность применения центральных разностей
if 0 < index_star < len(x) - 1:
    h = x[index_star + 1] - x[index_star]  # предполагаем равномерную сетку
    first_derivative = (y[index_star + 1] - y[index_star - 1]) / (2 * h)
    second_derivative = (y[index_star + 1] - 2 * y[index_star] + y[index_star - 1]) / (
        h**2
    )

    print(f"Первая производная в точке x* = {x_star}: {first_derivative}")
    print(f"Вторая производная в точке x* = {x_star}: {second_derivative}")

    output += f"Первая производная в точке x* = {x_star}: {first_derivative}\n"
    output += f"Вторая производная в точке x* = {x_star}: {second_derivative}\n"
else:
    print("Ошибка: Невозможно применить центральные разности в выбранной точке.")
    output += "Ошибка: Невозможно применить центральные разности в выбранной точке.\n"

# --- Сохранение вывода в файл ---
with open("output/3_4.txt", "w", encoding="utf-8") as file_out:
    file_out.write(output)

Таблица значений:
x = [-1.  0.  1.  2.  3.]
y = [-0.5      0.       0.5      0.86603  1.     ]

Первая производная в точке x* = 1.0: 0.433015
Вторая производная в точке x* = 1.0: -0.13397000000000003


## 3.5 Вычисление определенного интеграла

In [ ]:
import os

# --- Создание папки для вывода ---
os.makedirs("output", exist_ok=True)

# --- Исходные данные ---
output = ""

# Границы интегрирования
x0 = -1
xk = 1


# Функция для интегрирования
def f(x):
    return x / (2 * x + 5)


# Шаги
h1 = 0.5
h2 = 0.25


# --- Методы численного интегрирования ---
def rectangles_method(f, x0, xk, h):
    n = int((xk - x0) / h)
    result = 0
    for i in range(n):
        xi = x0 + i * h
        xi_mid = xi + h / 2
        result += f(xi_mid)
    return result * h


def trapezoids_method(f, x0, xk, h):
    n = int((xk - x0) / h)
    result = 0.5 * (f(x0) + f(xk))
    for i in range(1, n):
        xi = x0 + i * h
        result += f(xi)
    return result * h


def simpson_method(f, x0, xk, h):
    n = int((xk - x0) / h)
    if n % 2 != 0:
        raise ValueError("Для метода Симпсона число интервалов должно быть чётным")
    result = f(x0) + f(xk)
    for i in range(1, n):
        xi = x0 + i * h
        if i % 2 == 0:
            result += 2 * f(xi)
        else:
            result += 4 * f(xi)
    return result * h / 3


# --- Метод Рунге–Ромберга ---
def runge_romberg(I1, I2, p):
    return abs(I2 - I1) / (2**p - 1)


# --- Вычисления ---
methods = {
    "Прямоугольников": (rectangles_method, 2),
    "Трапеций": (trapezoids_method, 2),
    "Симпсона": (simpson_method, 4),
}

print("Результаты расчётов:\n")
output += "Результаты расчётов:\n\n"

for method_name, (method_func, p) in methods.items():
    I_h1 = method_func(f, x0, xk, h1)
    I_h2 = method_func(f, x0, xk, h2)
    error = runge_romberg(I_h1, I_h2, p)

    print(f"{method_name}:")
    print(f"  Интеграл с шагом h1 = {h1}: {I_h1}")
    print(f"  Интеграл с шагом h2 = {h2}: {I_h2}")
    print(f"  Оценка погрешности (Рунге–Ромберг): {error}\n")

    output += f"{method_name}:\n"
    output += f"  Интеграл с шагом h1 = {h1}: {I_h1}\n"
    output += f"  Интеграл с шагом h2 = {h2}: {I_h2}\n"
    output += f"  Оценка погрешности (Рунге–Ромберг): {error}\n\n"

# --- Сохранение результатов ---
with open("output/3_5.txt", "w", encoding="utf-8") as file_out:
    file_out.write(output)

Результаты расчётов:

Прямоугольников:
  Интеграл с шагом h1 = 0.5: -0.05450105450105448
  Интеграл с шагом h2 = 0.25: -0.05794799368631646
  Оценка погрешности (Рунге–Ромберг): 0.0011489797284206585

Трапеций:
  Интеграл с шагом h1 = 0.5: -0.06845238095238096
  Интеграл с шагом h2 = 0.25: -0.061476717726717735
  Оценка погрешности (Рунге–Ромберг): 0.0023252210752210747

Симпсона:
  Интеграл с шагом h1 = 0.5: -0.05952380952380953
  Интеграл с шагом h2 = 0.25: -0.05915149665149663
  Оценка погрешности (Рунге–Ромберг): 2.482085815419301e-05

